<a href="https://colab.research.google.com/github/254francis/Cryptography_class/blob/main/CTF_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Running exhaustive transforms to decode the puzzle.
# Steps:
# 1. Convert provided hex bytes to ASCII (this should form a Base64 string).
# 2. Base64-decode that to get a payload string.
# 3. Apply ROT47 to get the "ctf_flag{...}" form (as previous steps suggested).
# 4. Extract inner payload between braces and try multiple transforms on it:
#    - letter-only ROT (1..25)
#    - printable-ASCII rotation (1..94)
#    - Base64-alphabet rotation (shift 0..63)
#    - Atbash
#    - rail-fence decrypt (rails 2..14)
#    - reverse
#    - single-byte XOR (1..255)
#    - common substitution @->a and variations
# We will score candidates by presence of English words like 'awesome', 'you', 'are', 'flag', or by matching typical flag patterns.
import base64, string, itertools, math

hex_input = "4E 45 55 33 4D 44 63 39 4D 6A 68 4D 63 53 52 34 63 33 51 6B 4D 48 31 77 65 43 4E 2B 63 58 68 68 58 32 46 6B 4D 48 42 49 4E 6B 52 41 50 6A 59 77 51 44 38 32 54 67 3D 3D"
hex_input = hex_input.replace(" ","").replace("\n","")
b = bytes.fromhex(hex_input)
ascii_b64 = b.decode('ascii', errors='ignore')
print("Step 1: Hex -> ASCII (Base64 string):")
print(ascii_b64)
print()

# Base64 decode
decoded = base64.b64decode(ascii_b64)
try:
    decoded_text = decoded.decode('latin1')  # preserve bytes
except:
    decoded_text = str(decoded)
print("Step 2: Base64 decode -> bytes/ASCII (latin1):")
print(decoded_text)
print()

# Apply ROT47 as earlier produced ctf_flag
def rot47(s):
    out = []
    for ch in s:
        o = ord(ch)
        if 33 <= o <= 126:
            out.append(chr(33 + ((o - 33 + 47) % 94)))
        else:
            out.append(ch)
    return "".join(out)

rot47_text = rot47(decoded_text)
print("Step 3: ROT47 result:")
print(rot47_text)
print()

# Find braces payload if present
import re
m = re.search(r'\{([^}]*)\}', rot47_text)
payload = m.group(1) if m else rot47_text
print("Payload to transform:")
print(payload)
print()

# Helper transforms
alphabet_letters = string.ascii_lowercase + string.ascii_uppercase
def rot_letters(s, n):
    def rot_char(c):
        if 'a' <= c <= 'z':
            return chr((ord(c)-97 + n) % 26 + 97)
        if 'A' <= c <= 'Z':
            return chr((ord(c)-65 + n) % 26 + 65)
        return c
    return "".join(rot_char(c) for c in s)

def rot_printable(s, n):
    out = []
    for ch in s:
        o = ord(ch)
        if 32 <= o <= 126:
            out.append(chr(32 + ((o - 32 + n) % 95)))
        else:
            out.append(ch)
    return "".join(out)

# Base64 alphabet rotation
B64_ALPH = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
def rot_base64_alph(s, shift):
    trans = str.maketrans(B64_ALPH, B64_ALPH[shift:]+B64_ALPH[:shift])
    return s.translate(trans)

def atbash(s):
    res = []
    for c in s:
        if 'a' <= c <= 'z':
            res.append(chr(ord('z') - (ord(c)-ord('a'))))
        elif 'A' <= c <= 'Z':
            res.append(chr(ord('Z') - (ord(c)-ord('A'))))
        else:
            res.append(c)
    return "".join(res)

def rail_fence_decrypt(cipher, rails):
    if rails <= 1: return cipher
    # build rails pattern
    n = len(cipher)
    rail_lens = [0]*rails
    idx = 0
    direction = 1
    for i in range(n):
        rail_lens[idx] += 1
        if idx == 0:
            direction = 1
        elif idx == rails-1:
            direction = -1
        idx += direction
    # slice cipher into rails
    rails_str = []
    pos = 0
    for rlen in rail_lens:
        rails_str.append(cipher[pos:pos+rlen])
        pos += rlen
    # read zigzag
    res = []
    pointers = [0]*rails
    idx = 0
    direction = 1
    for i in range(n):
        res.append(rails_str[idx][pointers[idx]])
        pointers[idx] += 1
        if idx == 0:
            direction = 1
        elif idx == rails-1:
            direction = -1
        idx += direction
    return "".join(res)

def single_byte_xor(sbytes, key):
    return bytes([b ^ key for b in sbytes])

# We'll try many transforms and score plausibility
candidates = set()

# 1) letter ROTs 1..25 and their results
for n in range(1,26):
    cand = rot_letters(payload, n)
    candidates.add(("rot_letters", n, cand))

# 2) printable ASCII rotations 1..94
for n in range(1,95):
    cand = rot_printable(payload, n)
    candidates.add(("rot_printable", n, cand))

# 3) base64 alphabet rotations (0..63)
for n in range(1,64):
    cand = rot_base64_alph(payload, n)
    candidates.add(("rot_b64alph", n, cand))

# 4) Atbash
candidates.add(("atbash", 0, atbash(payload)))

# 5) rail-fence decryption 2..14
for r in range(2,15):
    cand = rail_fence_decrypt(payload, r)
    candidates.add(("rail_fence", r, cand))

# 6) reverse
candidates.add(("reverse", 0, payload[::-1]))

# 7) single-byte XOR on bytes of payload (try keys 1..127) - only if payload contains mostly non-binary
pbytes = payload.encode('latin1')
for k in range(1,128):
    xb = single_byte_xor(pbytes, k)
    try:
        s = xb.decode('latin1')
        candidates.add(("xor", k, s))
    except:
        pass

# 8) substitution @->a and variations applied to previous candidates to normalize
def apply_common_subs(s):
    subs = s.replace('@','a').replace('1','l').replace('/','_').replace('0','o').replace('5','s')
    return subs

augmented = set()
for method, param, cand in list(candidates):
    augmented.add((method, param, cand))
    augmented.add((method, param, apply_common_subs(cand)))
candidates = augmented

# Score candidates by simple heuristics (presence of English words)
keywords = ['awesome','you','are','flag','ctf','one','nice','the','team','congrats']
scored = []
for method,param,cand in candidates:
    score = 0
    low = cand.lower()
    for kw in keywords:
        if kw in low:
            score += 5
    # reward human-readable chars and underscores / braces
    readable_frac = sum(1 for ch in cand if ch.isprintable())/max(1,len(cand))
    score += readable_frac
    # reward length conformity (reasonable length)
    if 5 < len(cand) < 80:
        score += 0.5
    scored.append((score, method, param, cand))

scored_sorted = sorted(scored, key=lambda x: -x[0])

# show top 40 unique textual candidates
seen = set()
out = []
for score, method, param, cand in scored_sorted:
    key = cand
    if key in seen: continue
    seen.add(key)
    out.append((round(score,3), method, param, cand))
    if len(out) >= 40:
        break

import pandas as pd
df = pd.DataFrame(out, columns=['score','method','param','candidate'])
import caas_jupyter_tools as tools; tools.display_dataframe_to_user("Top candidates", df)
df.head(40)


Step 1: Hex -> ASCII (Base64 string):
NEU3MDc9MjhMcSR4c3QkMH1weCN+cXhhX2FkMHBINkRAPjYwQD82Tg==

Step 2: Base64 decode -> bytes/ASCII (latin1):
4E707=28Lq$xst$0}px#~qxa_ad0pH6D@>60@?6N

Step 3: ROT47 result:
ctf_flag{BSIDES_NAIROBI2025_Awesome_one}

Payload to transform:
BSIDES_NAIROBI2025_Awesome_one



ModuleNotFoundError: No module named 'caas_jupyter_tools'